In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as c
import astropy.units as u
from scipy.interpolate import interp2d
from scipy.optimize import curve_fit
from scipy.optimize import minimize

import emcee
import corner

import twopoppy
import dsharp_opac as op

import dipsy

from dipsy.cgs_constants import au, year, c_light, jy_sas

%matplotlib inline

## Load opacities

In [ ]:
opac = dipsy.Opacity()
rho_s_op = opac.rho_s

## Set up simulation

In [ ]:
args = twopoppy.args()

# make sure we use the same grain density as in the opacities

args.rhos = rho_s_op

In [ ]:
args.print_args()

In [ ]:
res = twopoppy.wrapper.model_wrapper(args)

In [ ]:
# set some time snapshot index
it = -1

In [ ]:
f, ax = plt.subplots()
ax.loglog(res.x / au, res.a_fr[it, :], label='fragmentation')
ax.loglog(res.x / au, res.a_dr[it, :], label='drift')
ax.loglog(res.x / au, res.a_t[it, :], 'k--', label='$a_\mathrm{max}$')
ax.set_ylim(1e-4, 1e4)
ax.set_title(f'time = {res.timesteps[it] / year:.2g} yr')
ax.legend();

In [ ]:
f, ax = plt.subplots()
ax.loglog(res.x / au, res.sigma_g[it, :], label='gas')
ax.loglog(res.x / au, res.sigma_d[it, :], label='dust')
ax.set_ylim(1e-4, 1e4)
ax.set_title(f'time = {res.timesteps[it] / year:.2g} yr')
ax.legend();

## Plot size distribution from the code

In [ ]:
f, ax = plt.subplots()
cc = ax.pcolormesh(res.x / au, res.a, np.log10(res.sig_sol), vmin=-10, vmax=1)
ax.set_xscale('log')
ax.set_yscale('log')
plt.colorbar(cc);

## Create power-law size distribution

In [ ]:
a, a_i, sig_da = dipsy.get_powerlaw_dust_distribution(res.sigma_d[it, :], res.a_t[it, :], a0=args.a0, na=100)

In [ ]:
f, ax = plt.subplots()
cc = ax.pcolormesh(res.x / au, a_i, np.log10(sig_da.T), vmin=-10, vmax=1)#, edgecolor='k')
ax.loglog(res.x / au, res.a_t[it, :], 'r')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim(a[[0, -1]])
#ax.set_xlim(2e-1, 4e-1)
#ax.set_ylim(6e0, 2e1)
plt.colorbar(cc);

## Calculate Intensity profiles

set wavelengths

In [ ]:
lam_obs = [0.087, 0.1, 0.3, 1]

Calculate observables

In [ ]:
obs = dipsy.get_observables(res.x, res.sigma_g[it], res.sigma_d[it], res.a_t[it], res.T, opac, lam_obs)

Plot the intensity profiles and the 68% radii

In [ ]:
f, ax = plt.subplots()
for _lam, _Inu, _rf in zip(lam_obs, obs.I_nu, obs.rf):
    l, = ax.loglog(res.x / au, _Inu, label=f'$\lambda = {_lam * 10:.2g}$ mm')
    ax.axvline(_rf / au, c=l.get_color(), ls='--')

ax.legend()
ax.set_xlim(1e0, 2e2);
ax.set_ylim(1e-3, 1e1);

## Fitting the dust line

In [ ]:
def truncated_powerlaw(x, y0, p, xout):
    return y0 * (x/x[0])**p * (x < xout)

In [ ]:
def lnprob(params, x, data, noise):
    if params[0]<1e-50:
        return -1e100
    if np.abs(params[1])>10:
        return -1e100
    if params[2]<x[0]:
        return -1e100
    if params[2]>x[-1]:
        return -1e100
    
    model = truncated_powerlaw(x, *params)
    rmsd = (data - model)**2
    # we ignore points that are too far away from the model
    #rmsd[rmsd>(100 * noise)**2] = 0.0
    rmsd /= 2 * noise**2

    return -rmsd.sum()

In [ ]:
noise = 1e-3
x     = np.linspace(res.x[0], 1e3 * au, 100)
data  = np.interp(x, res.x, obs.I_nu[0, :] + noise * np.random.randn(args.nr))
sigma = data * 0.05 + noise
p0    = [data[0], -0.5, 100 * au]

In [ ]:
nwalkers = 40
nburnin = 2000
nsteps = 10000
sampler = emcee.EnsembleSampler(nwalkers, len(p0), lnprob, args=[x, data, sigma])

inisamples = np.array([
    p0[0] * 10**(-1 + 2 * np.random.rand(nwalkers)),
    p0[1] + (-1 + 2 * np.random.rand(nwalkers)),
    10* au + 200 * au * np.random.rand(nwalkers),
    ]).T

# first burn in to keep the ones with reasonable acceptance fraction

burnin = sampler.run_mcmc(inisamples, nburnin)
good = inisamples[sampler.acceptance_fraction>0.25, :]
if len(good) == 0:
    good = inisamples[sampler.acceptance_fraction > np.sort(sampler.acceptance_fraction)[nwalkers//2], :]
inisamples = good[np.random.choice(np.arange(len(good)), size=nwalkers)]

# second burn in to keep the ones with higher probability

sampler.reset()
burnin = sampler.run_mcmc(inisamples, nburnin)
final_prob = sampler.lnprobability[:, -1]
good = np.arange(nwalkers)[final_prob > np.median(final_prob)]
inisamples = inisamples[np.random.choice(good, size=nwalkers)]

sampler.reset()
output = sampler.run_mcmc(inisamples, nsteps)

# get the final good samples
final_prob = sampler.lnprobability[:, -1]
good = np.arange(nwalkers)[final_prob > np.median(final_prob)]

Check convergence

In [ ]:
plt.loglog(np.arange(nsteps), -sampler.lnprobability.T);

In [ ]:
acc_time = sampler.get_autocorr_time()
discard = int(5 * acc_time.max())
flat_chain = sampler.chain[good, discard:, :].reshape(-1, 3)

In [ ]:
f, ax = plt.subplots()
line = ax.loglog(x / au, data, label='data')
ax.fill_between(x / au, data + sigma, np.maximum(1e-100, data - sigma), color=line[0].get_color(), alpha=0.3)

ax.loglog(x / au, truncated_powerlaw(x, *p0), 'k--', label='guess')

for sample in flat_chain[-100:,:]:
    ax.loglog(x / au, truncated_powerlaw(x, *sample), lw=0.5, alpha=0.5)

ax.set_ylim(1e-5, 1e2)
ax.legend();

In [ ]:
corner_data = flat_chain.copy()

corner_data[:, 0] = np.log10(corner_data[:, 0])
corner_data[:, 2] /= au
#corner_data[:, 3] = np.log10(corner_data[:, 3])

In [ ]:
corner.corner(corner_data, bins=100, range=[[0, 2], [-3, 3], [1, 200]]);

In [ ]:
p_mcmc = flat_chain.mean(0)
s_mcmc = flat_chain.std(0)

print(f'Σ0    = {p_mcmc[0]:.2f} +/- {s_mcmc[0]:.2f} g/cm^2')
print(f'p     = {p_mcmc[1]:.2f} +/- {s_mcmc[1]:.2f} g/cm^2')
print(f'r_out = {p_mcmc[-1] / au:.2f} +/- {s_mcmc[-1] / au:.2f} au')

#### Nelder-Mead

In [ ]:
def obj_func(params, x, data, sigma):
    return -lnprob(params, x, data, sigma)

opt_res = minimize(obj_func, p0, args=(x, data, sigma), method='Nelder-Mead', options={'disp':True})
p_nm = opt_res.x

#### LM Methods

In [ ]:
p_lm, cov = curve_fit(truncated_powerlaw, x, data, p0=p0, sigma=sigma)

In [ ]:
p_lm, cov = curve_fit(truncated_powerlaw, x, data, p0=p0, sigma=sigma, absolute_sigma=True)

In [ ]:
f, ax = plt.subplots()
ax.loglog(x / au, data, label='data')
ax.loglog(res.x / au, truncated_powerlaw(res.x, *p_mcmc), label='MCMC')
ax.loglog(res.x / au, truncated_powerlaw(res.x, *p_nm), '--', label='NM')
ax.loglog(res.x / au, truncated_powerlaw(res.x, *p_lm), '--', label='LM')
#ax.set_ylim(1e-6, 1e-5)
ax.legend();